In [ ]:
# import pandas as pd
# import geopandas as gpd
# from shapely.geometry import box

# # Read the CSV
# df = pd.read_csv("Coordinates_GEE.csv")

# # Create polygons from bounding box
# df['geometry'] = df.apply(lambda row: box(row['left'], row['bottom'], row['right'], row['top']), axis=1)

# # Convert to GeoDataFrame
# gdf = gpd.GeoDataFrame(df, geometry='geometry')

# # Set CRS to WGS84 (EPSG:4326)
# gdf.set_crs(epsg=4326, inplace=True)

# # Save as shapefile
# print(gdf.head())
# print(gdf.crs)
# print(gdf.total_bounds)
# gdf.to_file("bounding_boxes.shp")


        left      right       top    bottom  name  \
0 -63.210069 -63.164075 -0.532219 -0.578213     0   
1 -63.210069 -63.164075 -0.624207 -0.670200     1   
2 -63.210069 -63.164075 -0.670200 -0.716194     2   
3 -63.210069 -63.164075 -0.716194 -0.762188     3   
4 -63.210069 -63.164075 -0.756439 -0.802432     4   

                                            geometry  
0  POLYGON ((-63.16408 -0.57821, -63.16408 -0.532...  
1  POLYGON ((-63.16408 -0.6702, -63.16408 -0.6242...  
2  POLYGON ((-63.16408 -0.71619, -63.16408 -0.670...  
3  POLYGON ((-63.16408 -0.76219, -63.16408 -0.716...  
4  POLYGON ((-63.16408 -0.80243, -63.16408 -0.756...  
EPSG:4326
[-68.94673277  -8.11069501 -47.44382241   1.29563027]


In [ ]:
# import simplekml
# kml = simplekml.Kml()

# # Add polygons to KML
# for idx, row in df.iterrows():
#     coords = [
#         (row['left'], row['bottom']),
#         (row['left'], row['top']),
#         (row['right'], row['top']),
#         (row['right'], row['bottom']),
#         (row['left'], row['bottom'])  # close the polygon
#     ]
#     pol = kml.newpolygon(name=f"Box {idx+1}", outerboundaryis=coords)
#     # pol.style.polystyle.color = simplekml.Color.changealphaint(100, simplekml.Color.blue)
    
#     pol.style.polystyle.color = simplekml.Color.changealphaint(0, simplekml.Color.red)  # fill color
#     pol.style.linestyle.color = simplekml.Color.black  # outline color
#     pol.style.linestyle.width = 2

# # Save KML file
# kml.save("bounding_boxes.kml")

In [3]:
import geopandas as gpd

# Step 1: Read the shapefile
gdf = gpd.read_file('bounding_boxes.shp')

# Step 2: Reproject to equal-area projection
gdf_equal_area = gdf.to_crs('EPSG:6933')  # or other equal-area CRS

# Step 3: Union all geometries to handle overlaps
merged_geom = gdf_equal_area.unary_union

# Step 4: Calculate area (merged)
total_area_m2 = merged_geom.area

# Step 5: Convert to sq km
total_area_km2 = total_area_m2 / 1e6

print(f"Total unique area covered: {total_area_km2:.2f} square kilometers")

Total unique area covered: 25194.73 square kilometers


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21344\4242552110.py:10: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  merged_geom = gdf_equal_area.unary_union


In [3]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import box
import simplekml

df = pd.read_csv('Coordinates_GEE(in).csv')  # or use pd.read_excel, etc.

# Step 2: Create bounding box geometries
df['geometry'] = df.apply(lambda row: box(row['left'], row['bottom'], row['right'], row['top']), axis=1)

# Step 3: Convert to GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')
gdf.set_crs("EPSG:4326", inplace=True)  # Set your CRS here if known

# Step 4: Split into two GeoDataFrames
gdf_yes = gdf[gdf['correct'].str.lower() == 'yes']
gdf_no = gdf[gdf['correct'].str.lower() == 'no']

# Step 5: Save as shapefiles
gdf_yes.to_file('bounding_boxes_correct_yes.shp')
gdf_no.to_file('bounding_boxes_correct_no.shp')

In [4]:
def create_kml(df_subset, filename, color_hex):
    kml = simplekml.Kml()
    for _, row in df_subset.iterrows():
        coords = [
            (row['left'], row['bottom']),
            (row['left'], row['top']),
            (row['right'], row['top']),
            (row['right'], row['bottom']),
            (row['left'], row['bottom'])  # Close the polygon
        ]
        pol = kml.newpolygon(name=row['name'], outerboundaryis=coords)
        pol.style.polystyle.color = color_hex  # AABBGGRR (alpha-blue-green-red)
        pol.style.polystyle.outline = 1
    kml.save(filename)

# Step 3: Split by 'correct'
df_yes = df[df['correct'].str.lower() == 'yes']
df_no = df[df['correct'].str.lower() == 'no']

# Step 4: Create KMLs with different colors
# Red with 80% opacity → 'cc0000ff', Green with 80% opacity → 'cc00ff00'
create_kml(df_yes, 'bounding_boxes_correct_yes.kml', 'cc00ff00')  # Green
create_kml(df_no, 'bounding_boxes_correct_no.kml', 'cc0000ff')    # Red